# LLMs with Langchain
Langchain is a library that allows you to make working with LLMs easier. It has integrations with most of the main LLM providers, providing a consistent interface. This allows you to build applications using any LLM with the same code.

*Note that langchain looks for API keys in environment variable by default, so if you have loaded `GROQ_API_KEY` using `dotenv` then you don't need to explicitly state it.*


In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_groq import ChatGroq

## Invoking LLM with langchain
You can invoke Groq using langchain with a simple user prompt.

In [17]:
llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    # api_key="<your_api_key>" # not needed if stored as environment variable
)
completion = llm.invoke("what is the densest element?")
type(completion)

langchain_core.messages.ai.AIMessage

That is nice, it returns an `AIMessage` object. If we want the text output we can access it through the `.content`attribute.

In [18]:
completion.content

"The densest element is generally considered to be osmium (Os), with a density of 22.59 g/cm³. Osmium is a hard, brittle, blue-gray or blue-black transition metal in the platinum group, and it is the densest naturally occurring element known.\n\nOsmium has a very high atomic mass (190.23 u) and a relatively small atomic radius, which contributes to its high density. In fact, osmium is about twice as dense as lead, which is often considered a very dense metal.\n\nHere's a comparison of the densities of some common elements to put osmium's density into perspective:\n\n* Osmium (Os): 22.59 g/cm³\n* Lead (Pb): 11.34 g/cm³\n* Gold (Au): 19.3 g/cm³\n* Uranium (U): 19.1 g/cm³\n* Tungsten (W): 19.3 g/cm³\n\nIt's worth noting that there are some synthetic elements, such as hassium (Hs) and meitnerium (Mt), that have even higher densities than osmium. However, these elements are not found naturally and are only produced in small quantities in laboratories.\n\nOverall, osmium's extremely high den

# Output parsers

Langchain has a bunch out output parsers, that will format the output for you into various formats.

#### `StrOutputParser` will give you the completion formatted as a simple string

In [16]:
# StrOuputParser
parser = StrOutputParser()
parser.invoke(completion)

"The densest element is generally considered to be osmium (Os), with a density of 22.59 g/cm³. Osmium is a hard, brittle, blue-gray or blue-black transition metal in the platinum group, and it is the densest naturally occurring element known.\n\nOsmium has a very high atomic mass (190.23 u) and a relatively small atomic radius, which contributes to its high density. In fact, osmium is about twice as dense as lead, which is often considered a very dense metal.\n\nHere's a comparison of the densities of some common elements to put osmium's density into perspective:\n\n* Osmium (Os): 22.59 g/cm³\n* Lead (Pb): 11.34 g/cm³\n* Gold (Au): 19.3 g/cm³\n* Uranium (U): 19.1 g/cm³\n* Tungsten (W): 19.3 g/cm³\n\nIt's worth noting that there are some synthetic elements, such as hassium (Hs) and meitnerium (Mt), that have even higher densities than osmium. However, these elements are not found naturally and are only produced in small quantities in laboratories.\n\nOverall, osmium's extremely high den

### `JsonOutputParser` will extract the JSON from the completion and return as a dict

In [26]:
# even if you ask for json you'll get some extra words
completion = llm.invoke("Please return the following data in JSON format: name=Daisy, breed=border collie")
completion.content

'Here is the data in JSON format:\n\n```\n{\n  "name": "Daisy",\n  "breed": "border collie"\n}\n```'

In [27]:
parser = JsonOutputParser()
parsed_completion = parser.invoke(completion)
parsed_completion

{'name': 'Daisy', 'breed': 'border collie'}

In [28]:
# It's a dictionary!
parsed_completion["name"]

'Daisy'

## Langchain Chains
Langchain's chains are a powerful feature that allows us to string together any `Runnable` components to make a chain that we can invoke with a simple command. 

They are built up using LangChain Expression Language (LCEL), whose syntax involves connection components with a pipe operator `|`.

Here is a simple chain that generates a completion and formats the output as a string:

In [29]:
chain = llm | StrOutputParser()
chain.invoke("What is a python dictionary?")

"In Python, a dictionary (also known as a hash table or associative array) is a built-in data type that stores a collection of key-value pairs. It is a mutable data structure, meaning it can be modified after creation.\n\nA dictionary consists of:\n\n1. **Keys**: Unique strings or integers that serve as identifiers for each value.\n2. **Values**: The actual data associated with each key. Values can be of any data type, including strings, integers, lists, dictionaries, and more.\n\nHere's an example of a simple dictionary:\n```python\nperson = {'name': 'John', 'age': 30, ' occupation': 'Developer'}\n```\nIn this example, the dictionary `person` has three key-value pairs:\n\n* `name` is the key, and `John` is the value.\n* `age` is the key, and `30` is the value.\n* `occupation` is the key, and `Developer` is the value.\n\nYou can access and manipulate dictionary values using their corresponding keys. For example:\n```python\nprint(person['name'])  # Output: John\nperson['age'] = 31  # U

## Prompts and Prompt Templates
So far we have just been invoking the LLM with just plain text. No explicit user or system prompt. Langchain has a prompt templates to make this easy to do.


In [54]:
# messages are formatted as tuples (<message type>, "<message content>")
messages = [
    ("system", "You are a helpful assistant."), 
    ("human", "Tell me joke about baking.")
]
ChatPromptTemplate.from_messages(messages)

ChatPromptTemplate(input_variables=[], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Tell me joke about baking.'))])

The prompt template allows you to make your prompts more dynamic by passing varibales to them without needing to re-write the prompt. 

Anything enclosed in `{}` will be interpreted as an input variable

In [39]:
messages = [
    ("system", "You are a helpful assistant."), 
    ("human", "Tell me a {content_type} about {subject}.")
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt_template

ChatPromptTemplate(input_variables=['content_type', 'subject'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['content_type', 'subject'], template='Tell me {content_type} about {subject}.'))])

In [51]:
print(prompt_template.format(content_type="joke", subject="baking"))

System: You are a helpful assistant.
Human: Tell me a joke about baking.


In [53]:
# can also pass variables as a dictionary
print(prompt_template.format(content_type="short story", subject="electron"))

System: You are a helpful assistant.
Human: Tell me a short story about electron.


## Putting it all together
Let's make a versatile chain that uses variables to change the prompt.


In [59]:
# prompt template
messages = [
    ("system", "You are a {personality} assistant."), 
    ("human", "Tell me a {content_type} about {subject}.")
]
prompt_template = ChatPromptTemplate.from_messages(messages)

# llm
llm = ChatGroq(temperature=1, model="llama3-70b-8192")

# output parser
parser = StrOutputParser()

# Define the chain
chain = prompt_template | llm | parser

#### When invoking a chain, you must pass the prompt variables as a dictionary

In [60]:
completion = chain.invoke(
    {"content_type": "knock-knock joke", 
     "subject": "elephants",
     "personality": "friendly"}
)
print(completion)

Here's one:

Knock, knock!
Who's there?
Tusky.
Tusky who?
Tusky you later, I'm going to the jungle gym!


In [64]:
completion = chain.invoke(
    {"content_type": "news headline", 
     "subject": "escaped snakes",
     "personality": "serious"}
)
print(completion)

**BREAKING: "Ssss-scape in the City: Over 100 Venomous Snakes Slither Free from Reptile House After Storm Damages Enclosures; Residents Warned to Be Vigilant as Authorities Scramble to Contain Situation"**


In [63]:
completion = chain.invoke(
    {"content_type": "advertisement", 
     "subject": "toothpaste",
     "personality": "funny"}
)
print(completion)

Here's a silly advertisement about toothpaste:

**Introducing "Smile-O-Matic" Toothpaste - The Toothpaste That'll Make You Smile (Literally!)**

[Announcer voice] Are you tired of brushing your teeth with a toothpaste that's as boring as a lecture on crop rotation? Well, put down that snooze-fest toothpaste and pick up Smile-O-Matic Toothpaste, the most excitement-filled toothpaste on the market!

With its unique blend of sparkles, glitter, and a hint of cotton candy flavor (just kidding, it's minty fresh, of course!), Smile-O-Matic Toothpaste is guaranteed to make you smile while you brush. And we're not just talking about a tiny, forced smile - we're talking about a full-on, teeth-shining, face-hurting smile!

But wait, there's more! Our patented "Tooth-Tastic" formula not only whitens and strengthens your teeth, but also gives you a temporary (completely safe and non-permanent, don't worry!) sparkly smile that'll make you the envy of all your friends. Just imagine the Instagram self